In [18]:
import pg8000
import csv

conn = pg8000.connect(
    user="postgres",
    password="m10223104.",
    database="postgres",
    host="localhost",
    port=5432
)
cur = conn.cursor()

cur.execute("""
    create table if not exists phonebook (
        id serial primary key,
        first_name varchar(100),
        phone varchar(20)
    );
""")
conn.commit()

def insert_user(name, phone):
    cur.execute("insert into phonebook (first_name, phone) values (%s, %s);", (name, phone))
    conn.commit()
    print("added!")

def insert_from_csv(filename):
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cur.execute("insert into phonebook (first_name, phone) values (%s, %s);", (row[0], row[1]))
        conn.commit()
        print("data from csv loaded!")

def update_user(old_name, new_name=None, new_phone=None):
    if new_name:
        cur.execute("update phonebook set first_name = %s where first_name = %s;", (new_name, old_name))
    if new_phone:
        cur.execute("update phonebook set phone = %s where first_name = %s;", (new_phone, old_name))
    conn.commit()
    print("updated!")

def query_users(filter_text=None):
    if filter_text:
        cur.execute("select * from phonebook where first_name like %s or phone like %s;", (f"%{filter_text}%", f"%{filter_text}%"))
    else:
        cur.execute("select * from phonebook;")
    rows = cur.fetchall()
    for row in rows:
        print(row)

def delete_user(name_or_phone):
    cur.execute("delete from phonebook where first_name = %s or phone = %s;", (name_or_phone, name_or_phone))
    conn.commit()
    print("deleted!")

def main_menu():
    while True:
        print("\n phonebook menu:")
        print("1 - add user")
        print("2 - load from csv")
        print("3 - update data")
        print("4 - view all users")
        print("5 - search by name or phone")
        print("6 - delete by name or phone")
        print("0 - exit")
        
        choice = input("choice: ")

        if choice == "1":
            name = input("enter name: ")
            phone = input("enter phone number: ")
            insert_user(name, phone)

        elif choice == "2":
            filename = input("enter file name (users.csv): ")
            insert_from_csv(filename)

        elif choice == "3":
            old_name = input("old name: ")
            new_name = input("new name (enter - skip): ") or None
            new_phone = input("new phone number (enter - skip): ") or None
            update_user(old_name, new_name, new_phone)

        elif choice == "4":
            query_users()

        elif choice == "5":
            filter_text = input("enter filter (name or phone): ")
            query_users(filter_text)

        elif choice == "6":
            val = input("name or phone to delete: ")
            delete_user(val)

        elif choice == "0":
            print("bye!")
            break

        else:
            print("invalid input. try again.")

main_menu()


 phonebook menu:
1 - add user
2 - load from csv
3 - update data
4 - view all users
5 - search by name or phone
6 - delete by name or phone
0 - exit


choice:  2
enter file name (users.csv):  phonebook_example.csv


data from csv loaded!

 phonebook menu:
1 - add user
2 - load from csv
3 - update data
4 - view all users
5 - search by name or phone
6 - delete by name or phone
0 - exit


choice:  0


bye!


In [53]:
import pygame
import sqlite3
import random

# Настройки
WIDTH, HEIGHT = 600, 400
CELL_SIZE = 20
WHITE, BLACK, GREEN, RED = (255, 255, 255), (0, 0, 0), (0, 255, 0), (255, 0, 0)

# Уровни
levels = {
    1: {"speed": 10, "walls": []},
    2: {"speed": 15, "walls": [(10,10), (10,11), (11,10)]},
    3: {"speed": 20, "walls": [(5,5), (5,6), (5,7), (6,5), (7,5)]}
}

# --- БАЗА ДАННЫХ ---
def create_tables():
    conn = sqlite3.connect("snake_game.db")
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS user (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT UNIQUE NOT NULL
        )
    ''')
    c.execute('''
        CREATE TABLE IF NOT EXISTS user_score (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER,
            level INTEGER,
            score INTEGER,
            FOREIGN KEY (user_id) REFERENCES user(id)
        )
    ''')
    conn.commit()
    conn.close()

def get_user_data(username):
    conn = sqlite3.connect('snake_game.db')
    c = conn.cursor()
    c.execute("SELECT id FROM user WHERE username = ?", (username,))
    result = c.fetchone()

    if result:
        user_id = result[0]
        c.execute("SELECT level, score FROM user_score WHERE user_id = ? ORDER BY id DESC LIMIT 1", (user_id,))
        score_data = c.fetchone()
        conn.close()
        if score_data:
            return user_id, score_data[0], score_data[1]
        else:
            return user_id, 1, 0
    else:
        c.execute("INSERT INTO user (username) VALUES (?)", (username,))
        user_id = c.lastrowid
        conn.commit()
        conn.close()
        return user_id, 1, 0

def save_progress(user_id, level, score):
    conn = sqlite3.connect('snake_game.db')
    c = conn.cursor()
    c.execute("INSERT INTO user_score (user_id, level, score) VALUES (?, ?, ?)", (user_id, level, score))
    conn.commit()
    conn.close()

# --- ИГРА ---
def draw_walls(screen, walls):
    for wall in walls:
        pygame.draw.rect(screen, RED, pygame.Rect(wall[0]*CELL_SIZE, wall[1]*CELL_SIZE, CELL_SIZE, CELL_SIZE))

def game_loop(user_id, level, score):
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Snake Game")
    clock = pygame.time.Clock()
    font = pygame.font.SysFont(None, 24)

    snake = [(5, 5)]
    direction = (1, 0)
    apple = (random.randint(0, WIDTH//CELL_SIZE-1), random.randint(0, HEIGHT//CELL_SIZE-1))
    walls = levels[level]["walls"]
    speed = levels[level]["speed"]
    current_score = score

    paused = False
    running = True
    while running:
        if paused:
            screen.fill(BLACK)
            msg = font.render("PAUSED - Press any key to resume", True, WHITE)
            screen.blit(msg, (WIDTH//2 - 100, HEIGHT//2))
            pygame.display.flip()
            for event in pygame.event.get():
                if event.type == pygame.KEYDOWN:
                    paused = False
        else:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    save_progress(user_id, level, current_score)
                    running = False
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_UP: direction = (0, -1)
                    elif event.key == pygame.K_DOWN: direction = (0, 1)
                    elif event.key == pygame.K_LEFT: direction = (-1, 0)
                    elif event.key == pygame.K_RIGHT: direction = (1, 0)
                    elif event.key == pygame.K_p:
                        save_progress(user_id, level, current_score)
                        paused = True

            # Движение змейки
            new_head = (snake[0][0] + direction[0], snake[0][1] + direction[1])
            if new_head in snake or new_head in walls or not (0 <= new_head[0] < WIDTH // CELL_SIZE) or not (0 <= new_head[1] < HEIGHT // CELL_SIZE):
                save_progress(user_id, level, current_score)
                running = False
                break

            snake.insert(0, new_head)
            if new_head == apple:
                current_score += 10
                apple = (random.randint(0, WIDTH//CELL_SIZE-1), random.randint(0, HEIGHT//CELL_SIZE-1))
            else:
                snake.pop()

            # Отрисовка
            screen.fill(BLACK)
            draw_walls(screen, walls)
            for block in snake:
                pygame.draw.rect(screen, GREEN, pygame.Rect(block[0]*CELL_SIZE, block[1]*CELL_SIZE, CELL_SIZE, CELL_SIZE))
            pygame.draw.rect(screen, WHITE, pygame.Rect(apple[0]*CELL_SIZE, apple[1]*CELL_SIZE, CELL_SIZE, CELL_SIZE))

            # Счёт
            score_text = font.render(f"Level: {level}  Score: {current_score}", True, WHITE)
            screen.blit(score_text, (10, 10))
            pygame.display.flip()
            clock.tick(speed)

    pygame.quit()


# --- ЗАПУСК ---
if __name__ == "__main__":
    create_tables()
    username = input("Enter your username: ")
    user_id, level, score = get_user_data(username)
    print(f"Welcome back, {username}! Starting from level {level} with score {score}.")
    game_loop(user_id, level, score)

Enter your username:  ттт


Welcome back, ттт! Starting from level 1 with score 0.
